In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [2]:
transition_matrix = np.array([
    [0.3, 0.7, 0, 0],
    [0.4, 0, 0.6, 0],
    [0.25, 0, 0, 0.75],
    [1, 0, 0, 0]
])

state_signal = [0, 1, 1, 1]

In [3]:
def random_sequence(length, transition_matrix=transition_matrix, state_signal=state_signal):
    shape = transition_matrix.shape
    assert shape[0] == shape[1]
    
    state = np.random.randint(shape[0])
    sequence = [state_signal[state]]
    
    for _ in range(length-1): # state is updated at every step
        r = np.random.random()
        
        probs = np.sort(transition_matrix[state]) # sort transition probabilities ascending
        prob_int = np.cumsum(probs)  # take the cumulative sum to ensure the unit interval is spanned
        
        for i in range(len(prob_int)):
            if r > prob_int[i]:
                continue
            else:  
                state = np.where(transition_matrix[state] == probs[i])[0][0]
                break
                
        sequence.append(state_signal[state])
        
    return np.array(sequence)

In [4]:
def has_runs(sequence, run_len):  # detects runs of 1 of a given length
    s = 0
    
    for i in sequence:
        if i == 1:
            s += 1
        else:
            s = 0
            
        if s == run_len:
            return True
        
    return False

In [5]:
x = np.array([random_sequence(10) for _ in range(250)])  # shape: (250, 10)
y = keras.utils.to_categorical([has_runs(i, 3) for i in x])  # shape: (250, 2)

In [6]:
x_in = keras.layers.Input(shape=(10,))
x_out = keras.layers.Dense(15, activation="relu")(x_in)
x_out = keras.layers.Dense(2, activation="softmax")(x_out)

model = keras.models.Model(inputs=x_in, outputs=x_out)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 15)                165       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 32        
Total params: 197
Trainable params: 197
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer="SGD", loss="binary_crossentropy", metrics=['accuracy'])
model.fit(x, y, epochs=750, batch_size=15, verbose=False)

In [8]:
test = np.expand_dims(np.array([1,1,1,0,0,0,1,1,0,1]), axis=0)  # this is annoying
model.predict(test)

array([[0.03216824, 0.96783173]], dtype=float32)